In [1]:
import pandas as pd
import numpy as np
from calc import calc_x_cy,calc_y_cy

colnames={
'DATE':'date',
'TIME':'time',
'BOROUGH':'bor',
'ZIP CODE':'zip',
'LATITUDE':'lat',
'LONGITUDE':'long',
'LOCATION':'loc',
'ON STREET NAME':'onstreet',
'CROSS STREET NAME':'crossstreet',
'OFF STREET NAME':'offstreet',
'NUMBER OF PERSONS INJURED':'n_inj',
'NUMBER OF PERSONS KILLED':'n_kill',
'NUMBER OF PEDESTRIANS INJURED':'ped_inj',
'NUMBER OF PEDESTRIANS KILLED':'ped_kill',
'NUMBER OF CYCLIST INJURED':'cyc_inj',
'NUMBER OF CYCLIST KILLED':'cyc_kill',
'NUMBER OF MOTORIST INJURED':'mot_inj',
'NUMBER OF MOTORIST KILLED':'mot_kill',
'CONTRIBUTING FACTOR VEHICLE 1':'factor1',
'CONTRIBUTING FACTOR VEHICLE 2':'factor2',
'CONTRIBUTING FACTOR VEHICLE 3':'factor3',
'CONTRIBUTING FACTOR VEHICLE 4':'factor4',
'CONTRIBUTING FACTOR VEHICLE 5':'factor5',
'UNIQUE KEY':'key',
'VEHICLE TYPE CODE 1':'vtype1',
'VEHICLE TYPE CODE 2':'vtype2',
'VEHICLE TYPE CODE 3':'vtype3',
'VEHICLE TYPE CODE 4':'vtype4',
'VEHICLE TYPE CODE 5':'vtype5'
}

y_sep=20
x_sep=10
# 40.6466779 40.804213 delta=0.157535
# -74.0171345 -73.9298911 delta=0.08724399

# lat_min,lat_max=40.645,40.805 #0.16
# long_min,long_max=-74.020,-73.925 #0.095

lat_min,lat_max=40.645,40.965 #0.16
long_min,long_max=-74.020,-73.830 #0.095

lat_sep=20
long_sep=20
lat_div=(lat_max-lat_min)/lat_sep
long_div=(long_max-long_min)/long_sep

In [2]:
col='collisions/NYPD_Motor_Vehicle_Collisions.csv'

df=pd.read_csv(col,skipinitialspace=True)#,nrows=100000)
df.rename(columns=colnames,inplace=True)
# print(df[0:10])

df=df[df.lat.between(30,50)];df=df[df.long.between(-90,-60)]
df=df[df.long>long_min]
print (df.lat.min(),df.lat.max())

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (22,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


40.5543879 40.912884


In [3]:
df['timestamp']=df[['date','time']].apply(lambda row:' '.join(row),axis=1)
df['timestamp']=pd.to_datetime(df['timestamp'],infer_datetime_format=True)
df['yr']=df.timestamp.dt.year
df['month']=df.timestamp.dt.month
df['hr']=df.timestamp.dt.hour
df['minu']=df.timestamp.dt.minute
df['weekday']=df.timestamp.dt.weekday
df['weekend']=0
df.loc[(df.weekday==5)|(df.weekday==6),'weekend']=1
df['time']=df.hr*60+df.minu

df['grid_x']=calc_x_cy(df['long'].values)
df['grid_y']=calc_y_cy(df['lat'].values)
df['accident_count']=1

# print (df[0:5])
# print (df[0:5])

In [4]:
# df['time_prd']=df.hr.astype(str)+'hr'
df['time_prd']=df.hr

df['inj_kill']=df.n_inj+df.n_kill

In [ ]:
print (len(df))
print (len(df[df.n_inj>1][['n_inj','ped_inj','cyc_inj','mot_inj']]))

### Accident count by each yr/month by each time_prd, by citibike or not

In [ ]:
df1=pd.pivot_table(df,index=['grid_x','grid_y','month','yr','weekend','bor'],values=['accident_count'],
                   columns='time_prd',aggfunc=np.sum,fill_value=0)
# print(df1[0:5])
df1.columns=df1.columns.droplevel()
# print(df1[0:5])
df1.reset_index(level=[0,1,2,3,4,5],inplace=True)
# print (df1[0:5])
citi_grid=pd.read_csv('sum/citibike_grid.csv')
# print (citi_grid[0:5])
df1=df1.merge(citi_grid,on=['grid_x','grid_y'],how='outer')
df1=df1[(df1.grid_x!=-1)&(df1.grid_y!=-1)]
df1.loc[df1.citigrid.isnull(),'citigrid']=0
# print(df1[0:5])
df1=df1.groupby(['month','yr','citigrid','weekend','bor'],as_index=False).agg('sum')
# print(df1[0:5])
df1['cat']=df1.apply(lambda row: '_'.join((str(int(row.yr)),str(int(row.month)),
                                           str(int(row.citigrid)),str(int(row.weekend)))),axis=1)
print (df1[0:5])

del df1['count'];del df1['grid_x'];del df1['grid_y']
df1.to_csv('sum/col_by_time.csv',index=False)


### Get the mean accident occur time for morning peaks

In [12]:
df1=df[(df.hr<=10)&(df.hr>=6)]
df1=df1[df1.yr==2013]
df1=df1[(df1.month==7)|(df1.month==8)|(df1.month==5)|(df1.month==6)]
# df1=df1[df1.weekend==1]

df1=df1.groupby(['grid_x','grid_y','month','yr'],as_index=False) \
                .agg({'time':'mean','accident_count':'count','n_inj':'sum','n_kill':'sum'})
df.to_csv('sum/a.csv',index=False)
# df1=pd.pivot_table(df1,index=['grid_x','grid_y','month'],values=['time'],columns='yr')
# df1.columns=df1.columns.droplevel()
# df1.reset_index(level=[0,1,2],inplace=True)
print (df1.columns)
# df1.rename(columns={2013:'2013',2012:'2012',2014:'2014',2015:'2015'},inplace=True)
print (df1.columns)
# df1['13_12']=df1['2013']-df1['2012']
# df1['14_12']=df1['2014']-df1['2012']
# df1['15_12']=df1['2015']-df1['2012']

# df1=df1.groupby(['month','yr'],as_index=False).agg({'time':'mean','accident_count':'count'})
# df1=pd.pivot_table(df1,index=['month','yr'],values=['time'],columns='yr')


#  merge wtih trip_summary
df_t=pd.read_csv('sum/trip_sum.csv')
df_t=df_t[['grid_x','grid_y','yr','month','age','count','duration']]
df_t=df_t[(df_t.yr==2013)&(df_t.month<=8)&(df_t.month>=5)]

df1=pd.merge(df1,df_t,on=['grid_x','grid_y','yr','month'],how='left')

## merge with citigrid and clustering
citi_grid=pd.read_csv('sum/citibike_grid.csv')
df1=df1.merge(citi_grid,on=['grid_x','grid_y'],how='outer')

cluster=pd.read_csv('sum/cluster.csv')
cluster=cluster[['grid_x','grid_y','cluster']]
print (cluster.columns)
df1=df1.merge(cluster,on=['grid_x','grid_y'],how='left')
# del df1['count']



df1=df1[df1.grid_x!=-1];df1=df1[df1.grid_y!=-1]
# df1.loc[df1.citigrid.isnull(),'citigrid']=0
# df1=df1[(df1['2012'].notnull())&(df1['2013'].notnull())&(df1['2014'].notnull())]
# print (df1[0:5])

df1.to_csv('sum/col_time.csv',index=False)

Index(['grid_x', 'grid_y', 'month', 'yr', 'time', 'accident_count', 'n_inj',
       'n_kill'],
      dtype='object')
Index(['grid_x', 'grid_y', 'month', 'yr', 'time', 'accident_count', 'n_inj',
       'n_kill'],
      dtype='object')
Index(['grid_x', 'grid_y', 'cluster'], dtype='object')


### Get 2012 to 2013_6 data for kmeans clustering to extract similar grids

In [ ]:

df1=df[((df.yr==2013)&(df.month<=6))|(df.yr==2012)]
# df1=df1[df1.month<=6]
df2=pd.pivot_table(df1,index=['grid_x','grid_y'],
                   values=['count'],columns='hr',fill_value=0,aggfunc=np.sum)
df2.columns=df2.columns.droplevel()
df2.reset_index(level=[0,1],inplace=True)
# print (df2[0:50])
df2.to_csv('2013.csv')

### Get 2012 to 2014_7 data and summarize into a single csv

In [ ]:

df2=df[(df.yr==2012)|(df.yr==2013)|((df.yr==2014)&(df.month<8))]
columns_used=['grid_x','grid_y','yr','month',\
            'n_inj', 'n_kill', 'ped_inj', 'ped_kill', \
            'cyc_inj', 'cyc_kill', 'mot_inj', 'mot_kill','accident_count']
df2=df2[columns_used]
# df2['accident_cnt']=1
print(df2[0:5])
print (len(df2))
df2=df2.groupby(['yr','month','grid_x','grid_y']).agg(['sum'])
print (len(df2))
df2.columns=df2.columns.droplevel(1)
df2.reset_index(level=[0,1,2,3],inplace=True)
df2=df2[(df2.grid_x!=-1)&(df2.grid_y!=-1)]
df2.to_csv('sum/col_sum.csv',index=False)
# print (df2[0:10])

## 3. Merge Collisions data and CitiBike trip data

In [ ]:
df_trip=pd.read_csv('sum/trip_sum.csv')
df_col=pd.read_csv('sum/col_sum.csv')
df=pd.merge(df_trip,df_col,on=['grid_x','grid_y','yr','month'],how='outer')

citi_grid=pd.read_csv('sum/citibike_grid.csv')
df=df.merge(citi_grid,on=['grid_x','grid_y'],how='outer')
df.loc[df.citigrid.isnull(),'citigrid']=0

cluster=pd.read_csv('sum/cluster.csv')
cluster=cluster[['grid_x','grid_y','cluster']]
df=df.merge(cluster,on=['grid_x','grid_y'],how='outer')

df.to_csv('sum/trip_col_sum.csv',index=False)

# x=pd.merge(cluster,citi_grid,on=['grid_x','grid_y'],how='outer')
# x.to_csv('sum/x.csv',index=False)
# # df.loc[(df.CitiBike.isnull()),'CitiBike']=0


### Get all the clusters

In [ ]:
citi_grid=pd.read_csv('sum/citibike_grid.csv')

clus=[]
for i in range(1,8):
    filename='clu'+str(i)+'.csv'
    clu=pd.read_csv('sum/'+filename)
    clu['cluster']=i
    clus.append(clu)
cluster=pd.concat(clus)

citi_grid=pd.read_csv('sum/citibike_grid.csv')
cluster=cluster.merge(citi_grid,on=['grid_x','grid_y'],how='outer')
cluster.loc[cluster.citigrid.isnull(),'citigrid']=0
del cluster['count']

cluster.to_csv('sum/cluster.csv',index=False)
# print (cluster)


### Note: Backup code, no use yet

In [ ]:
df_in = df[(df.grid_x!=-1)&(df.grid_y!=-1)]
df_out= df[(df.grid_x==-1)]#|(df.grid_y==-1)]

# print (df_in[0:4])
# print (df_out[0:4])

# df1=df[(df.month==5)&(df.yr==2013)]
# df2=df[(df.month==5)&(df.yr==2014)]

In [ ]:
## average accident occur time
df_in_temp=df_in.groupby(['yr','month','time_prd','weekend'],as_index=False).agg({'count':'count'})
# df_out_temp=df_out.groupby(['yr','month'],as_index=False).agg({'time':'mean'})
df_in_temp.to_csv('df_in_temp.csv')
# df_out_temp.to_csv('df_out_temp.csv')

In [ ]:
# print(df1[0:10])
df_in_temp=pd.pivot_table(df_in, index=['yr','month','weekend'],values=['count'],columns='time_prd',fill_value=0,aggfunc=np.sum)
df_in_temp.columns=df_in_temp.columns.droplevel()
df_in_temp.to_csv('df_in_temp.csv')

print (df_in_temp[0:5])

df_out_temp=pd.pivot_table(df_out, index=['yr','month'],values=['count'],columns='time_prd',fill_value=0,aggfunc=np.sum)
df_out_temp.columns=df_out_temp.columns.droplevel()
df_out_temp.to_csv('df_out_temp.csv')
print (df_out_temp[0:5])

In [ ]:
# df3=df[(df.yr==2015)&(df.month==10)]
# df3.to_csv('2015_10.csv')

In [ ]:
# print (dir(df.timestamp.dt))
# print (df.timestamp.dt.weekday_name)